###Connect to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###configuration file

In [22]:
def get_config():
    return {
        "model":"T-CLM",
        "logs": "/content/drive/MyDrive/Colab Notebooks/T-CLM_log/T-CLM",
        "batch_size": 8,
        "num_epochs": 30,
        "lr": 1e-4,
        "seq_len": 512,
        "d_model": 768,
        "n_layers": 12,
        "head": 12,
        "d_ff": 3072,
        "dropout": 0.1,
        "masking_prob": 0.15,
        "vocab_size": 50261,
        "model_file_path": "/content/drive/MyDrive/Colab Notebooks/T-CLM/T-CLM.pt",
        "tokenizer_file": "/content/drive/MyDrive/Colab Notebooks/T-CLM/tokenizer.json",
    }


###BPE Tokenizer


In [3]:
from pathlib import Path
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

def get_all_sentences(ds, field):
    for item in ds:
        yield item[field]

def build_or_get_tokenizer(config, ds):
    tokenizer_path = Path(config['tokenizer_file'])
    if not tokenizer_path.exists():
        tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
        trainer = trainers.BpeTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]", "[MASK]"], min_frequency=1)
        tokenizer.train_from_iterator(get_all_sentences(ds, "text"), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))

    return tokenizer

###Data Pipeline

In [4]:
import torch
from torch.utils.data import DataLoader, random_split
import json
from torch.utils.data import Dataset

class BilingualDataset(Dataset):
    def __init__(self, ds, tokenizer, seq_len):
        self.seq_len = seq_len
        self.ds = ds
        self.tokenizer = tokenizer
        self.sos_token = torch.tensor([tokenizer.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer.token_to_id("[PAD]")], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        text = self.ds[idx]['text']
        input_tokens = self.tokenizer.encode(text).ids

        # Truncate if too long
        if len(input_tokens) > self.seq_len - 2:
            input_tokens = input_tokens[:self.seq_len - 2]

        num_padding_tokens = self.seq_len - len(input_tokens) - 2

        input = torch.cat(
            [
                self.sos_token,
                torch.tensor(input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # The label is shifted right by one
        label = torch.cat(
            [
                torch.tensor(input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * (num_padding_tokens + 1), dtype=torch.int64),
            ],
            dim=0,
        )

        assert input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "input": input,
            "label": label,
            "mask": (input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
            "text": text,
        }


###Load dataset

In [5]:
def get_ds(config):
    with open('/content/drive/MyDrive/Colab Notebooks/T-CLM/dataset.json', 'r', encoding='utf-8') as f:
        ds_raw = json.load(f)

    # ds_raw = load_dataset(f"bookcorpus/bookcorpus", f"plain_text", split='train', trust_remote_code=True)

    tokenizer = build_or_get_tokenizer(config, ds_raw)
    train_ds_size = int(0.95 * len(ds_raw))
    val_ds_size = len(ds_raw) - train_ds_size
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

    train_ds = BilingualDataset(train_ds_raw, tokenizer, config['seq_len'])
    val_ds = BilingualDataset(val_ds_raw, tokenizer, config['seq_len'])

    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

    return train_dataloader, val_dataloader, tokenizer

In [19]:
config = get_config()
train_dataloader, val_dataloader, tokenizer = get_ds(config)

# for batch in train_dataloader:
#   print(batch['input'][0])
#   print(batch['label'][0])
#   break

print(tokenizer.get_vocab_size())

50261


###Casual masking

In [14]:
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

###Transformer model

In [15]:
import torch
import torch.nn as nn
import math

class LayerNormalization(nn.Module):
    def __init__(self, features: int, eps: float = 1e-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features))
        self.bias = nn.Parameter(torch.zeros(features))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

class FeedForwardBlock(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.embedding.embedding_dim)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.shape[1], :].requires_grad_(False)
        return self.dropout(x)

class ResidualConnection(nn.Module):
    def __init__(self, features: int, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization(features)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.h = h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h
        self.w_q = nn.Linear(d_model, d_model, bias=False)
        self.w_k = nn.Linear(d_model, d_model, bias=False)
        self.w_v = nn.Linear(d_model, d_model, bias=False)
        self.w_o = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1)
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # Combine all the heads together
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        return self.w_o(x)


class DecoderBlock(nn.Module):
    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

class Decoder(nn.Module):
    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class ProjectionLayer(nn.Module):
    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        return self.proj(x)

class Transformer(nn.Module):
    def __init__(self, decoder: Decoder, tgt_embed: InputEmbeddings, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.decoder = decoder
        self.tgt_embed = tgt_embed
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def decode(self, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, tgt_mask)

    def project(self, x):
        return self.projection_layer(x)

def build_transformer(tgt_vocab_size: int, tgt_seq_len: int, d_model: int, N: int, h: int, dropout: float, d_ff: int) -> Transformer:
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    transformer = Transformer(decoder, tgt_embed, tgt_pos, projection_layer)

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer


In [41]:
config = get_config()

model = build_transformer(config['vocab_size'], config['seq_len'], config['d_model'], config['n_layers'], config['head'], config['dropout'], config['d_ff'])
print(model)

# for name, param in model.named_parameters():
#     print(name, param)

Transformer(
  (decoder): Decoder(
    (layers): ModuleList(
      (0-11): 12 x DecoderBlock(
        (self_attention_block): MultiHeadAttentionBlock(
          (w_q): Linear(in_features=768, out_features=768, bias=False)
          (w_k): Linear(in_features=768, out_features=768, bias=False)
          (w_v): Linear(in_features=768, out_features=768, bias=False)
          (w_o): Linear(in_features=768, out_features=768, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward_block): FeedForwardBlock(
          (linear_1): Linear(in_features=768, out_features=3072, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (residual_connections): ModuleList(
          (0-1): 2 x ResidualConnection(
            (dropout): Dropout(p=0.1, inplace=False)
            (norm): LayerNormalization()
          )
        )
      )
    )
    (norm): LayerNormalizati

In [38]:
config = get_config()
device = "cuda" if torch.cuda.is_available() else "cpu"

model = build_transformer(config['vocab_size'], config['seq_len'], config['d_model'], config['n_layers'], config['head'], config['dropout'], config['d_ff'])

train_dataloader, _, _ = get_ds(config)

for batch in train_dataloader:
    input_ids = batch['input'][0].to(device)
    labels = batch['label'][0].to(device)
    break

print("Input shape: ",input_ids.shape)

mask = torch.triu(torch.ones(config['seq_len'], config['seq_len']), diagonal=1)  # Upper triangular matrix
# print(mask)
mask = mask.unsqueeze(0)

print("Mask shape: ", mask.shape)

with torch.no_grad():  # No need to calculate gradients
    output = model.decode(input_ids, mask)

print("Output shape: ",output.shape)

# Project the output to the vocabulary size
logits = model.project(output)

print("Logits.shape: ",logits.shape)

loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id('[PAD]'), label_smoothing=0.1).to(device)
loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
print("loss: ",loss.item())

Input shape:  torch.Size([512])
Mask shape:  torch.Size([1, 512, 512])
Output shape:  torch.Size([1, 512, 768])
Logits.shape:  torch.Size([1, 512, 50261])
loss:  10.775079727172852


###Validation run

In [ ]:
def run_validation(model, val_dataloader, tokenizer, seq_len, device, log_fn, global_step, writer):
    model.eval()
    total_loss = 0.0
    num_batches = len(val_dataloader)
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input'].to(device)
            labels = batch['label'].to(device)

            # Create causal mask
            seq_len = input_ids.size(1)
            causal_mask = torch.tril(torch.ones((seq_len, seq_len), device=device)).unsqueeze(0).unsqueeze(0)

            logits = model.decode(input_ids, causal_mask)
            logits = model.project(logits)

            logits = logits.view(-1, logits.size(-1))
            labels = labels.view(-1)

            loss = loss_fn(logits, labels)
            total_loss += loss.item()

    avg_loss = total_loss / num_batches
    perplexity = math.exp(avg_loss)
    print(f"Validation | Avg Loss: {avg_loss:.4f} | Perplexity: {perplexity:.2f}")

    writer.add_scalar('Validation/Loss', avg_loss, global_step)
    writer.add_scalar('Validation/Perplexity', perplexity, global_step)

    model.train()


In [ ]:
def get_weights_file_path(config):
    model_file_path = config.get('model_file_path', '')
    if Path(model_file_path).exists():
        return str(model_file_path)
    else:
        return None

###Get model

In [ ]:
def get_decoder_only_model(config, vocab_size):
    model = build_transformer(vocab_size, config['seq_len'], config['d_model'], config['n_layers'], config['head'], config['dropout'], config['d_ff'])
    return model


###Preload model

In [ ]:
def load_model(config, device, tokenizer, optimizer):
    if os.path.exists(config['model_file_path']):
        checkpoint = torch.load(config['model_file_path'], map_location=device)
        model = get_decoder_only_model(config, tokenizer.get_vocab_size()).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        global_step = checkpoint.get('global_step', 0)
        print(f"Loaded checkpoint from epoch {epoch}, global_step {global_step}")
        return model, epoch, global_step
    else:
        print("No checkpoint found. Starting training from scratch.")
        model = get_decoder_only_model(config, tokenizer.get_vocab_size()).to(device)
        return model, 0, 0


###Training loop

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import warnings
import os
import sys
import json
from pathlib import Path
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torchmetrics

def train(config):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)
    if device == 'cuda':
        print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
        print(f"Device memory: {torch.cuda.get_device_properties(device).total_memory / 1024 ** 3:.2f} GB")
    device = torch.device(device)

    train_dataloader, val_dataloader, tokenizer = get_ds(config)
    model = get_decoder_only_model(config, tokenizer.get_vocab_size()).to(device)
    writer = SummaryWriter(config['logs'])
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'], eps=1e-8)
    model, initial_epoch, global_step = load_model(config, device, tokenizer, optimizer)
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

    # Initialize the learning rate scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    for epoch in range(initial_epoch, config['num_epochs']):
        torch.cuda.empty_cache()
        model.train()
        batch_iterator = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{config['num_epochs']}")
        total_loss = 0.0
        num_batches = len(train_dataloader)

        for batch in batch_iterator:
            input_ids = batch['input'].to(device)  # (batch_size, seq_len)
            labels = batch['label'].to(device)        # (batch_size, seq_len)

            seq_len = input_ids.size(1)   # Shape: (1, 1, seq_len, seq_len)
            causal_mask = torch.tril(torch.ones((seq_len, seq_len), device=device)).unsqueeze(0).unsqueeze(0)

            optimizer.zero_grad(set_to_none=True)

            # Forward pass through the decoder-only model
            logits = model.decode(input_ids, causal_mask)  # (batch_size, seq_len, vocab_size)
            logits = model.project(logits)                 # (batch_size, seq_len, vocab_size)

            # Reshape logits and labels for computing loss
            logits = logits.view(-1, logits.size(-1))      # (batch_size * seq_len, vocab_size)
            labels = labels.view(-1)                       # (batch_size * seq_len)

            loss = loss_fn(logits, labels)

            loss.backward()


            # Optional: Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            global_step += 1
            writer.add_scalar('Train/Loss', loss.item(), global_step)
            batch_iterator.set_postfix({'Loss': loss.item()})

        # End of epoch logging
        avg_loss = total_loss / num_batches
        writer.add_scalar('Train/Average_Loss', avg_loss, epoch + 1)

        # Validation
        run_validation(model, val_dataloader, tokenizer, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step, writer)

        scheduler.step()

        # Save the model checkpoint
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'global_step': global_step,
        }, config['model_file_path'])

    writer.close()

if __name__ == '__main__':
    warnings.filterwarnings("ignore")
    config = get_config()
    train(config)


NameError: name 'get_config' is not defined

###Tensorboard

In [ ]:
%reload_ext tensorboard

import tensorflow as tf
import tensorboard

log_dir = "/content/drive/MyDrive/Colab Notebooks/T-CLM/T-CLM/"
%tensorboard --logdir {log_dir}